# References
- https://github.com/junaidnasirkhan/Restaurant-Visitor-Forecasting/blob/master/Restaurant%20Visitor%20Forecasting.ipynb
- https://github.com/ligz08/Kaggle-Recruit-Restaurant-Visitor-Forecasting

## System
- Hot Pepper Gourmet(hpg): 이용자들이 식당을 찾고 온라인 예약할 수 있는 시스템
- AirREGI / Restaurant Board(air): 예약 관리 및 계산 시스템

## air_store_info.csv
식당에서 air 시스템에 등록한 정보
- air_store_id: 식당 ID(air)
- air_genre_name: 식당의 종류(ex.이탈리안, 이자카야) 
- air_area_name: 지역
- latitude: 경도 (해당 지역의 위도, 경도)
- longitude: 위도

## hpg_store_info.csv
식당에서 hpg 시스템에 등록한 정보
- air_store_id: 식당 ID(hpg)
- air_genre_name: 식당의 종류(ex.이탈리안, 이자카야) 
- air_area_name: 지역
- latitude: 경도 (해당 지역의 위도, 경도)
- longitude: 위도

## air_reserve.csv
- air_store_id: 식당의 ID(air)
- visit_datetime: 예약 시간(방문 예정 시간)
- reserve_datetime: 예약 등록을 한 시간
- reserve_visitors: 예정 방문자 수

## hpg_reserve.csv
- hpg_store_id: 식당의 ID(hpg)
- visit_datetime: 예약 시간(방문 예정 시간)
- reserve_datetime: 예약 등록을 한 시간
- reserve_visitors: 예정 방문자 수

## store_id_relation.csv
두 시스템에서의 ID가 매칭되어 있다.
- hpg_store_id
- air_store_id

## air_visit_data.csv
방문 기록이 담겨있다.
- air_store_id: 식당의 ID(air)
- visit_date - 방문 날짜
- visitors - 방문자의 수

## date_info.csv
- calendar_date: 날짜(년, 월, 일)
- day_of_week: 요일
- holiday_flg - 일본 기준 휴일 여부(1,0)

## sample_submission.csv
제출 양식
- id: air 기준 식당 ID와 방문 날짜가 결합된 형태
- visiotors: id에 해당하는 날짜, 식당에 방문할 손님의 수 예측


# 데이터 불러오기 및 확인

In [161]:
import pandas as pd
import matplotlib.pyplot as plt

air_reserve = pd.read_csv("air_reserve.csv")
hpg_reserve = pd.read_csv("hpg_reserve.csv")
air_store_info = pd.read_csv("air_store_info.csv")
hpg_store_info = pd.read_csv("hpg_store_info.csv")
store_id_relation = pd.read_csv("store_id_relation.csv")
air_visit_data = pd.read_csv("air_visit_data.csv")
date_info = pd.read_csv("date_info.csv")
stores_to_pred = pd.read_csv("sample_submission.csv")


In [162]:
stores_to_pred

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0
...,...,...
32014,air_fff68b929994bfbd_2017-05-27,0
32015,air_fff68b929994bfbd_2017-05-28,0
32016,air_fff68b929994bfbd_2017-05-29,0
32017,air_fff68b929994bfbd_2017-05-30,0


## air_store_info

In [163]:
air_store_info.shape

(829, 5)

In [164]:
air_store_info.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


In [165]:
# 중복값 확인
air_store_info['air_store_id'].duplicated().sum()

0

## hpg_store_info

In [166]:
hpg_store_info.shape

(4690, 5)

In [167]:
hpg_store_info.head()

,hpg_store_id,hpg_genre_name,hpg_area_name,latitude,longitude
0,hpg_6622b62385aec8bf,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
1,hpg_e9e068dd49c5fa00,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
2,hpg_2976f7acb4b3a3bc,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
3,hpg_e51a522e098f024c,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
4,hpg_e3d0e1519894f275,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221


air 시스템에는 829개, hpg 시스템에는 4690개의 식당이 등록되어 있습니다.

## air_reserve

In [168]:
air_reserve.shape

(92378, 4)

In [169]:
air_reserve['air_store_id'].nunique()

314

In [170]:
air_reserve.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5


## hpg_reserve

In [171]:
hpg_reserve.shape

(2000320, 4)

In [172]:
hpg_reserve.head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13


## store_id_relation

In [173]:
store_id_relation.shape

(150, 2)

In [174]:
store_id_relation.head()

,air_store_id,hpg_store_id
0,air_63b13c56b7201bd9,hpg_4bc649e72e2a239a
1,air_a24bf50c3e90d583,hpg_c34b496d0305a809
2,air_c7f78b4f3cba33ff,hpg_cd8ae0d9bbd58ff9
3,air_947eb2cae4f3e8f2,hpg_de24ea49dc25d6b8
4,air_965b2e0cf4119003,hpg_653238a84804d8e7


## air_visit_data

In [175]:
air_visit_data.shape

(252108, 3)

In [176]:
air_visit_data.head()

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


## date_info

In [177]:
date_info.head()

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


## stores to pred

In [178]:
stores_to_pred.head()

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0


In [179]:
id_split = stores_to_pred['id'].str.split('_', expand=True)
id_split.columns = ['source', 'store_id', 'visit_date']
stores_to_pred = pd.concat([stores_to_pred, id_split], axis=1)
stores_to_pred['air_store_id']=stores_to_pred['source'].str.cat(stores_to_pred['store_id'], sep='_')
stores_to_pred = stores_to_pred[['id', 'air_store_id', 'visit_date', 'visitors']]
stores_to_pred['visit_date'] = pd.to_datetime(stores_to_pred['visit_date'])

stores_to_pred.head()

,id,air_store_id,visit_date,visitors
0,air_00a91d42b08b08d9_2017-04-23,air_00a91d42b08b08d9,2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,air_00a91d42b08b08d9,2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,air_00a91d42b08b08d9,2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,air_00a91d42b08b08d9,2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,air_00a91d42b08b08d9,2017-04-27,0


# full <- air_visits + stores_to_pred

In [180]:
air_visit_data['id'] = air_visit_data['air_store_id'].str.cat(air_visit_data['visit_date'].astype(str), sep='_')

In [181]:
full = pd.concat([air_visit_data, stores_to_pred], 
                 axis=0, keys=['past','future'], names=['dataset'])\
        .reset_index(level='dataset')\
        .reset_index(drop=True)

In [182]:
display(full.info())
display(full.head())
display(full.tail())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284127 entries, 0 to 284126
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   dataset       284127 non-null  object
 1   air_store_id  284127 non-null  object
 2   visit_date    284127 non-null  object
 3   visitors      284127 non-null  int64 
 4   id            284127 non-null  object
dtypes: int64(1), object(4)
memory usage: 10.8+ MB


None

,dataset,air_store_id,visit_date,visitors,id
0,past,air_ba937bf13d40fb24,2016-01-13,25,air_ba937bf13d40fb24_2016-01-13
1,past,air_ba937bf13d40fb24,2016-01-14,32,air_ba937bf13d40fb24_2016-01-14
2,past,air_ba937bf13d40fb24,2016-01-15,29,air_ba937bf13d40fb24_2016-01-15
3,past,air_ba937bf13d40fb24,2016-01-16,22,air_ba937bf13d40fb24_2016-01-16
4,past,air_ba937bf13d40fb24,2016-01-18,6,air_ba937bf13d40fb24_2016-01-18


,dataset,air_store_id,visit_date,visitors,id
284122,future,air_fff68b929994bfbd,2017-05-27 00:00:00,0,air_fff68b929994bfbd_2017-05-27
284123,future,air_fff68b929994bfbd,2017-05-28 00:00:00,0,air_fff68b929994bfbd_2017-05-28
284124,future,air_fff68b929994bfbd,2017-05-29 00:00:00,0,air_fff68b929994bfbd_2017-05-29
284125,future,air_fff68b929994bfbd,2017-05-30 00:00:00,0,air_fff68b929994bfbd_2017-05-30
284126,future,air_fff68b929994bfbd,2017-05-31 00:00:00,0,air_fff68b929994bfbd_2017-05-31


In [183]:
import os
if not os.path.exists('scratch'): os.mkdir('scratch')

full.to_csv('scratch/full.csv', index=False)

# Merge Stores

In [184]:
# hpg_store_info를 사용하기 위해 air_sotre_id와 병합
hpg_stores = pd.merge(hpg_store_info, store_id_relation, how='left', on='hpg_store_id')
hpg_stores.head()

,hpg_store_id,hpg_genre_name,hpg_area_name,latitude,longitude,air_store_id
0,hpg_6622b62385aec8bf,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221,NaN
1,hpg_e9e068dd49c5fa00,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221,NaN
2,hpg_2976f7acb4b3a3bc,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221,NaN
3,hpg_e51a522e098f024c,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221,NaN
4,hpg_e3d0e1519894f275,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221,NaN


In [185]:
# air_sotre_id가 없으면 train에 쓸 수 없으므로 nan값이 있는 행 제거
hpg_stores.dropna(subset=['air_store_id'], inplace=True)

# hpg_store_id는 필요 없으므로 제거 
hpg_stores.drop(columns=['hpg_store_id'], inplace=True)
hpg_stores.shape

(63, 5)

63개의 추가 데이터를 얻을 수 있습니다.

In [186]:
#air_store_info와 합쳐주기 위해 column 순서와 이름을 변경
col = hpg_stores.columns[:-1].to_list()
hpg_stores = hpg_stores.reindex(columns=['air_store_id']+col)
hpg_stores.rename(columns={'hpg_genre_name':'air_genre_name', 'hpg_area_name':'air_area_name'}, inplace=True)

In [187]:
# 모든 store 정보 합쳐주기
stores = pd.concat([air_store_info, hpg_stores], axis=0)

In [188]:
stores

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
...,...,...,...,...,...
4520,air_8f3b563416efc6ad,Creative Japanese food,Tōkyō-to Shinjuku-ku None,35.691384,139.701256
4581,air_52a08ef3efdb4bb0,Karaoke,Hokkaidō Asahikawa-shi 3 Jōdōri,43.768033,142.359664
4616,air_2a485b92210c98b5,Cafe,Hiroshima-ken Hiroshima-shi Hondōri,34.392106,132.461914
4645,air_0a74a5408a0b8642,Steak/Hamburger/Curry,Hokkaidō Sapporo-shi Minami 4 Jōnishi,43.055047,141.353236


위와 같은 방식으로 합쳐진 stores를 사용하려 했으나..!

In [189]:
stores.duplicated().sum()

0

In [190]:
stores['air_store_id'].duplicated().sum()

63

전체 중복은 없지만 id의 중복은 있는 즉, **같은 id를 가지면서 다른 정보를 담고 있는 데이터**들이 생겼습니다.

## relation 파일의 문제점

In [191]:
air_store_info[air_store_info['air_store_id']=='air_c47aa7493b15f297']

,air_store_id,air_genre_name,air_area_name,latitude,longitude
458,air_c47aa7493b15f297,Izakaya,Hiroshima-ken Hiroshima-shi Kokutaijimachi,34.386244,132.455018


In [192]:
hpg_store_info[hpg_store_info['hpg_store_id']=='hpg_03795a62e1c5f1c3']

,hpg_store_id,hpg_genre_name,hpg_area_name,latitude,longitude
178,hpg_03795a62e1c5f1c3,Japanese style,Hiroshima-ken Hiroshima-shi Hondōri,34.392106,132.461914


In [193]:
store_id_relation[store_id_relation['hpg_store_id']=='hpg_03795a62e1c5f1c3']

,air_store_id,hpg_store_id
113,air_c47aa7493b15f297,hpg_03795a62e1c5f1c3


위의 결과들을 보면 relation 파일에서 매칭되어 있는 id들임에도 불구하고 **air 시스템과 hpg 시스템에서 등록된 정보가 상이**합니다. Japanese style과 Izakaya는 표기 상의 차이로 보이고 지역의 차이는 시스템 상에서 선택할 수 있는 지역이 얼마나 세분화되어 있느냐의 차이이거나 특정 지역들이 어떤 시스템에서는 존재하지 않아 가까운 지역을 선택하여 등록하는 등의 문제로 예상됩니다. relation을 활용할 수 없으니 어쩔 수 없이 **stores는 air 시스템의 정보만 이용**하겠습니다.

In [194]:
air_store_info.to_csv('scratch/air_store_info.csv', index=False)

# Merge Reservation

In [195]:
# hpg_reserve에 air_id 연결시킨 후 필요한 column만 남기기
hpg_reserve = pd.merge(hpg_reserve, store_id_relation, on='hpg_store_id')[air_reserve.columns]

In [196]:
# air_reserve와 합치기
reserves = pd.concat([air_reserve, hpg_reserve], axis=0)
reserves.shape

(120561, 4)

In [97]:
reserves.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5


In [98]:
reserves.to_csv('scratch/reserves.csv', index=False)